# Warning: Unfinished Document. Work in Progress.

In [ ]:
%pip install particle
%pip install sympy
%pip install phasespace

Lets start by defining a decay. In the code beow you can see how to define a simple utility, which given momenta will be able to calculate all needed rotations.

For simplicity we will choose a simple
As you see, the isobars are always isolated, meaning no isobar is the predecessor of another isobar. This makes the the full amplitude straightforward to calculate.

In [ ]:
from decayangle.decay_topology import TopologyCollection, Node, Topology
from decayangle.config import config as cfg
cfg.sorting = "off"
# We can switch the backend to jax if we like, but this is usually not advisable
# Angles should be precomputed and thus jax is slower than numpy in most cases
# cfg.backend="jax"
# print(cfg.backend)

tg = TopologyCollection(
    topologies=[
        Topology(0, ((2,3), 1)),
        Topology(0, ((3,1), 2)),
        Topology(0, ((1,2), 3)),
    ]
)
for decay_topology in tg.topologies:
    print(decay_topology)

First lets define some basic utility functions. This is not something you need to understand 100% on the first try.
What we do is set up the basic functional setup to be able to perform a three body analysis, which is the simplest case. 

In [ ]:
from typing import NamedTuple, Callable
from math import prod
from particle import Particle
import numpy as np

def wigner_small_d(theta, j, m1, m2):
    """Calculate Wigner small-d function. Needs sympy.
      theta : angle
      j : spin (in units of 1/2, e.g. 1 for spin=1/2)
      m1 and m2 : spin projections (in units of 1/2)

    :param theta:
    :param j:
    :param m1: before rotation
    :param m2: after rotation

    """
    from sympy import Rational
    from sympy.abc import x
    from sympy.utilities.lambdify import lambdify
    from sympy.physics.quantum.spin import Rotation as Wigner

    j, m1, m2 = int(j), int(m1), int(m2)
    d = Wigner.d(Rational(j, 2), Rational(m1, 2), Rational(m2, 2), x).doit().evalf()
    d = lambdify(x, d, "numpy")(theta)
    # d = np.array(d)
    # d[np.isnan(d)] = 0
    d = np.nan_to_num(d, copy=True, nan=0.0)
    return d


def wigner_capital_d(phi, theta, psi, j, m1, m2):
    return (
        np.exp(-1j * phi * m1 / 2)
        * wigner_small_d(theta, j, m1, m2)
        * np.exp(-1j * psi * m2 / 2)
    )

def BWResonance(spin, mass, width):
    """Create a Breit-Wigner resonance function for a given spin.
    Args:
        spin (int): spin quantum number multiplied by 2
    """
    def f(s, L):
        return np.sqrt(s) / (s - mass**2 + 1j * mass * width)
    return f

class resonance:
    def __init__(self, spin, parity, mass, width, name, mother:Particle, bachelor:Particle, isobar: tuple):
        self.spin = spin
        self.mass = mass
        self.width = width
        self.lineshape = BWResonance(spin, mass, width)
        self.parity = parity
        self.name = name
        self.mother = mother
        self.bachelor = bachelor
        self.isobar = isobar


    @property
    def possible_helicities(self):
        return list(range(-self.spin, self.spin+1, 2))
    
    @property
    def isbar_spins(self):
        return [p.spin for p in self.isobar]
    
    @property
    def isobar_parities(self):
        return [p.parity for p in self.isobar]

    def LS_couplings_mother_decay(self):
        allowed_ls = []
        LSCoupling = NamedTuple('LSCoupling', [('L', int), ('S', int), ('coupling', complex)])
        for L in range(0, self.mother.spin + self.spin + self.bachelor.spin + 6, 2):
            for S in range(abs(self.spin - self.bachelor.spin), abs(self.spin + self.bachelor.spin) + 1, 2):
                if abs(L - S) <= self.mother.spin <= L + S:
                    allowed_ls.append((L, S))
        return {
            f"{self.name}:motherDecay:L{L}.S{S}": LSCoupling(L, S, 1 + 0j) for L, S in allowed_ls
        }
    
    def LS_coupling_resonance_decay(self):
        allowed_ls = []
        LSCoupling = NamedTuple('LSCoupling', [('L', int), ('S', int), ('coupling', complex)])
        for L in range(0, self.spin + sum(self.isbar_spins) + self.mother.spin, 2):
            for S in range(abs(self.isbar_spins[0] - self.isbar_spins[1]), sum(self.isbar_spins) + 1, 2):
                if abs(L - S) <= self.spin <= L + S: # check J double
                    if prod(self.isobar_parities) * (-1)**(L) == self.parity: # check parity
                        allowed_ls.append((L, S))
        return {
            f"{self.name}:resonanceDecay:L{L}.S{S}": LSCoupling(L, S, 1 + 0j) for L, S in allowed_ls
        }
            
    def clebsch_gordan(self, j1, m1, j2, m2, J, M):
        """
        Return clebsch-Gordan coefficient. Note that all arguments should be multiplied by 2
        (e.g. 1 for spin 1/2, 2 for spin 1 etc.). Needs sympy.
        """
        from sympy.physics.quantum.cg import CG
        from sympy import Rational
        cg = CG(
                Rational(j1, 2),
                Rational(m1, 2),
                Rational(j2, 2),
                Rational(m2, 2),
                Rational(J, 2),
                Rational(M, 2),
            ).doit().evalf()
        cg = float(cg)
        if str(cg) == "nan":
            raise ValueError(f"CG({j1/2},{m1/2},{j2/2},{m2/2},{J/2},{M/2}) is not a number")
        return cg
    
    def helicity_coupling_times_lineshape(self, s, hi_, hj_, convention="helicity"):
        ls_resonance_decay = self.LS_coupling_resonance_decay()
        pi, pj = self.isobar
        h = sum(
            ls.coupling * self.lineshape(s, ls.L) * 
            self.clebsch_gordan(pi.spin, hi_, pj.spin, -hj_, ls.S,hi_ - hj_) * 
            self.clebsch_gordan(ls.L, 0, ls.S, hi_ - hj_, self.spin, hi_ - hj_)
             * (ls.L + 1) ** 0.5
            / (self.spin + 1) ** 0.5
            for ls in ls_resonance_decay.values()
        )
        if convention == "minus_phi":
            # TODO insert phase difference here
            pass

        return h * (-1)**((pj.spin - hj_) / 2)

    
    
    def h_mother(self,hk_, hiso_, convention="helicity") -> float:
        """
        Calculate the helicity amplitude for the mother particle decay.
        The phase is (-1)**(pk.spin - hk_) comes as a consequence of the definition of the two particle state.
        H = (-1)**(pk.spin - hk_) H_tilde
        H_tilde = sum_{ls} coupling_ls * CG(pi.spin, hi_, pj.spin, -hj_, ls.S, hi_ - hj_) * CG(ls.S, hi_ - hj_, ls.L, 0, J, hi_ - hj_)
        H_tilde = 
        """
        mother_decay = self.LS_couplings_mother_decay()
        pk = self.bachelor
        h= sum(
            ls.coupling *  self.clebsch_gordan(self.spin, hiso_, pk.spin, -hk_, ls.S, hiso_ - hk_) * self.clebsch_gordan(ls.L, 0, ls.S, hiso_ - hk_, self.mother.spin, hiso_ - hk_)
            * (ls.L + 1) ** 0.5
            / (self.mother.spin + 1) ** 0.5
            for ls in mother_decay.values()
        ) 
        
        if convention == "minus_phi":
            # TODO insert phase difference here
            pass
        
        return h* (-1)**(pk.spin - hk_) 

possible_helicities = lambda spin: list(range(-spin, spin+1, 2))

The particle porperites can be fetched directly from the PDG, with the help of the particles project.

We go for a decay of $\Lambda_b \rightarrow \Lambda_c^+ \overline{D}^0 K^-$

In [ ]:
# shortcut to get the pdg parameters for our particles
class particle:
    def __init__(self, p):
        self.p = p 
    
    @property
    def spin(self):
        return int(self.p.J * 2)

    @property
    def parity(self):
        return self.p.P

    def __getattr__(self, attr):
        return getattr(self.p, attr)
    
    def helicities(self):
        return possible_helicities(self.spin)

p0 = particle(Particle.findall('Lambda(b)0')[0])
p1 = particle(Particle.findall('D0')[0])
p2 = particle(Particle.findall('Lambda(c)+')[0])
p3 = particle(Particle.findall('K-')[0])

particles = {
    0: p0,
    1: p1,
    2: p2,
    3: p3
}

for i in range(4):
    print(f"Particle {i}: {particles[i].name} spin: {particles[i].spin} helicities: {particles[i].helicities()}")


Now we have the main components for the amplitude. We can describe the lineshape for as single resonance, we can compute helicity couplings and we can calculate all the angles. Only thing left to do is write the loops for the sums. The final form should look somewhat like this:

$\sum_{\mathrm{topologys}}\sum_{\{\lambda\}}\sum_{\{\lambda^{'}\}} R(m,\{\lambda^{'}\}) d(\theta, \lambda_k^{'}, \lambda_{ij}^{'}) d(\phi_1,\lambda^{'}_1,\lambda_1)d(\phi_2,\lambda^{'}_2,\lambda_2)d(\phi_3,\lambda^{'}_3,\lambda_3)$

This simply describes the sum over all different possible decay topologys, and the sum over all possible helicities.
A second sum over the primes helicities $\{\lambda^{'}\}$ in conjunction with the wigner rotation $d(\theta, \lambda_k^{'}, \lambda_{ij}^{'})$ describes the needed basis change after the rotation to move the $(ij)$ isobar onto the z axis.

The rotations $d(\phi_1,\lambda^{'}_1,\lambda_1)d(\phi_2,\lambda^{'}_2,\lambda_2)d(\phi_3,\lambda^{'}_3,\lambda_3)$ describe the rotations rotating the final states of the different chains to a common constellation.

The $R(m,\{\lambda^{'}\})$ is the lineshape of the resonance together with the couplings $H_{0 \rightarrow (ij)\, k}^{\lambda_0, \lambda_{(ij) }- \lambda_k} \cdot X(m,L) \cdot H_{(ij) \rightarrow i\, j}^{\lambda_{(ij)}, \lambda_i^{'} - \lambda_j^{'}}$. 
The couplings $H$ are called helicity couplings 

In [ ]:
import phasespace as phsp

def generate_momenta(n=1_000):
    weights, p = phsp.nbody_decay(particles[0].mass, [
    particles[1].mass,
    particles[2].mass,
    particles[3].mass
    ]).generate(n, seed=1)


    momenta = {
        i + 1: np.array(p[f"p_{i}"])
            for i, _ in enumerate( [1,2,3])
    }
    return momenta, weights

momenta, weights = generate_momenta(10_000)


Next we want to define the structure of the amplitude

In [ ]:
reference_topology = tg.topologies[0]
momenta = reference_topology.to_rest_frame(momenta)
momenta = reference_topology.align_with_daughter(momenta)

# We can define resonances based on the isobar they can appear in 
resonance_lineshapes = {
    (1, 3): [ 
                resonance(0, 1, 2800, 30, 'D2800', p0, p2, (p1, p3)),
                # resonance(1, 1, 2800, 80, 'D2800', p0, p2, (p1, p3)) 
            ], 
    (2 ,3): [
                resonance(1, -1, 2900, 20, 'Xi_c', p0, p1, (p2, p3)),
                resonance(3, -1, 3200, 20, 'Xi_c2', p0, p1, (p2, p3)),
                # resonance(5, -1, 3600, 20, 'Xi_c2', p0, p1, (p2, p3)),
                # resonance(5, -1, 3500, 20, 'Xi_c2', p0, p1, (p2, p3)),
                # resonance(5, -1, 3100, 20, 'Xi_c2', p0, p1, (p2, p3)),

            ]   
    }

Now we need to get the values for our rotations. For this we can simply iterate through all the possible decay topologys, the ```TopologyCollection``` Object generates for us.

In [ ]:
for topology in tg.topologies:
        final_state_rotations = reference_topology.relative_wigner_angles(topology, momenta)
        isobars = topology.helicity_angles(momenta)

topo = tg.topologies[0]
h = topo.helicity_angles(momenta)

For each topology, we see, that we get exactly one isobar constellation. We also see, that of the two angles we compute, only one is non 0 (or $\pi$).

This is expected, as we are already in the decay plane. Thus this angle should go to 0. Computing it could be left out.

In [ ]:
reference_topology

In [ ]:
from functools import partial

# particle 1 
spin1 = p1.spin
helicities1 = possible_helicities(spin1)

# particle 2
spin2 = p2.spin
helicities2 = possible_helicities(spin2)

# particle 3
spin3 = p3.spin
helicities3 = possible_helicities(spin3)

spin0 = p0.spin
helicities0 = possible_helicities(spin0)

# precompute the angles
final_state_rotations = {
    topology.tuple: topology.relative_wigner_angles(reference_topology, momenta)
    for topology in tg.topologies
}
helicity_angles = {
    topology.tuple: topology.helicity_angles(momenta)
    for topology in tg.topologies
}
def f(h0, h1, h2, h3, resonance_lineshapes=resonance_lineshapes):
    # print("Helicity")
    helicity_list = [h0, h1, h2, h3]
    spin_list = [spin0, spin1, spin2, spin3]
    amplitude = 0
    for topology in tg.topologies:
        final_state_rotation = final_state_rotations[topology.tuple]
        isobars = helicity_angles[topology.tuple]
        for (isobar, bachelor), (phi, theta) in isobars.items():
            if isobar not in resonance_lineshapes:
                # guard clause against key errors
                continue
            # print(isobar, phi, theta, isobars[isobar].theta_rf, isobars[isobar].phi_rf)
            # determinethe correct helicities
            (i,j), k = isobar, bachelor
            hi ,hj, hk = helicity_list[i], helicity_list[j], helicity_list[k]
            si, sj, sk = spin_list[i], spin_list[j], spin_list[k]
            
            theta_ij = isobars[isobar].theta_rf
            phi_ij = isobars[isobar].phi_rf

            amplitude += sum(
                (resonance.spin + 1) ** 0.5
                * resonance.helicity_coupling_times_lineshape(
                    topology.nodes[isobar].mass(momenta) ** 2, hi_, hj_
                )
                * np.conj(
                    wigner_capital_d(phi, theta, 0, spin0, h0, h_iso - hk_)
                )  # mother decay
                * np.conj(
                    wigner_capital_d(
                        phi_ij, theta_ij, 0, resonance.spin, h_iso, hi_ - hj_
                    )
                )  # isobar decay
                * np.conj(wigner_capital_d(*final_state_rotation[i], si, hi_, hi))
                * np.conj(wigner_capital_d(*final_state_rotation[j], sj, hj_, hj))
                * np.conj(wigner_capital_d(*final_state_rotation[k], sk, hk_, hk))
                * resonance.h_mother(hk_, h_iso)
                for resonance in resonance_lineshapes.get(isobar, [])
                for h_iso in resonance.possible_helicities
                for hk_ in particles[bachelor].helicities()
                for hi_ in particles[i].helicities()
                for hj_ in particles[j].helicities()
            )
    return amplitude


In [ ]:
# precompute the angles
final_state_rotations_minus_phi = {
    topology.tuple: topology.relative_wigner_angles(reference_topology, momenta, convention='minus_phi')
    for topology in tg.topologies
}
helicity_angles_minus_phi = {
    topology.tuple: topology.helicity_angles(momenta, convention='minus_phi')
    for topology in tg.topologies
}

def f_minus_phi(h0, h1, h2, h3, resonance_lineshapes=resonance_lineshapes):
    # print("Minus Phi")
    helicity_list = [h0, h1, h2, h3]
    spin_list = [spin0, spin1, spin2, spin3]
    amplitude = 0
    for topology in tg.topologies:
        final_state_rotation = final_state_rotations_minus_phi[topology.tuple]
        isobars = helicity_angles_minus_phi[topology.tuple]
        for (isobar, bachelor), (phi, theta) in isobars.items():
            if isobar not in resonance_lineshapes:
                # guard clause against key errors
                continue
            # print(isobar, phi, theta, isobars[isobar].theta_rf, isobars[isobar].phi_rf)
            # determinethe correct helicities
            (i,j), k = isobar, bachelor
            hi ,hj, hk = helicity_list[i], helicity_list[j], helicity_list[k]
            si, sj, sk = spin_list[i], spin_list[j], spin_list[k]
            
            theta_ij = isobars[isobar].theta_rf
            phi_ij = isobars[isobar].phi_rf

            amplitude += sum(
                (resonance.spin + 1) ** 0.5
                * resonance.helicity_coupling_times_lineshape(
                    topology.nodes[isobar].mass(momenta) ** 2, hi_, hj_
                )
                * np.conj(
                    wigner_capital_d(phi, theta, -phi, spin0, h0, h_iso - hk_)
                )  # mother decay
                * np.conj(
                    wigner_capital_d(
                        phi_ij, theta_ij, -phi_ij, resonance.spin, h_iso, hi_ - hj_
                    )
                )  # isobar decay
                * np.conj(wigner_capital_d(*final_state_rotation[i], si, hi_, hi))
                * np.conj(wigner_capital_d(*final_state_rotation[j], sj, hj_, hj))
                * np.conj(wigner_capital_d(*final_state_rotation[k], sk, hk_, hk))
                * resonance.h_mother(hk_, h_iso)
                for resonance in resonance_lineshapes.get(isobar, [])
                for h_iso in resonance.possible_helicities
                for hk_ in particles[bachelor].helicities()
                for hi_ in particles[i].helicities()
                for hj_ in particles[j].helicities()
            )
    return amplitude


In [ ]:
import matplotlib.pyplot as plt
from decayangle.kinematics import mass_squared

m2_12 = mass_squared(momenta[1] + momenta[2])**0.5
m2_13 = mass_squared(momenta[1] + momenta[3])**0.5
m2_23 = mass_squared(momenta[2] + momenta[3])**0.5


In [ ]:
resonance_lineshapes_1 = {
    (3, 1): [ 
                resonance(0, 1, 2800, 30, 'D2800', p0, p2, (p1, p3)),
                resonance(1, 1, 2800, 80, 'D2800', p0, p2, (p1, p3)) 
            ], 
    (2 ,3): [
                # resonance(1, -1, 2900, 20, 'Xi_c', p0, p1, (p2, p3)),
                # resonance(3, -1, 3200, 20, 'Xi_c2', p0, p1, (p2, p3)),
                # resonance(5, -1, 3600, 20, 'Xi_c2', p0, p1, (p2, p3)),
                # resonance(5, -1, 3500, 20, 'Xi_c2', p0, p1, (p2, p3)),
                # resonance(5, -1, 3100, 20, 'Xi_c2', p0, p1, (p2, p3)),

            ]   
    }

resonance_lineshapes_2 = {
    (3, 1): [ 
                resonance(0, 1, 2800, 100, 'D2800', p0, p2, (p1, p3)),
                # resonance(2, 1, 2800, 80, 'D2800', p0, p2, (p1, p3)) ,
                # resonance(2, 1, 2850, 80, 'D2800', p0, p2, (p1, p3)) ,
                # resonance(0, 1, 2800, 80, 'D2800', p0, p2, (p1, p3)) ,
                # resonance(2, 1, 2800, 80, 'D2800', p0, p2, (p1, p3)) ,
            ], 
    (2 ,3): [
                resonance(1, -1, 2900, 100, 'Xi_c', p0, p1, (p2, p3)),
                # resonance(3, -1, 3200, 60, 'Xi_c2', p0, p1, (p2, p3)),
                # resonance(5, -1, 3600, 40, 'Xi_c2', p0, p1, (p2, p3)),
                # resonance(5, -1, 3500, 60, 'Xi_c2', p0, p1, (p2, p3)),
                # resonance(5, -1, 3100, 20, 'Xi_c2', p0, p1, (p2, p3)),

            ],
    (1, 2): [
            resonance(3, -1, 4500, 100, 'Pc', p0, p3, (p1, p2)),

        ]   
    }

def amp_dict(func, resonances):
    return {
        (l1, l2, l3, l4): func(l1, l2, l3, l4, resonance_lineshapes=resonances)
        for l1 in helicities0
        for l2 in helicities1
        for l3 in helicities2
        for l4 in helicities3
    }

def unpolarized(func, resonances):
    return sum(
        abs(v)**2 for v in amp_dict(func, resonances).values()
    )




In [ ]:
a1_1 = amp_dict(f, resonance_lineshapes_2)
a1_2 = amp_dict(f_minus_phi, resonance_lineshapes_2)

In [ ]:
from matplotlib.colors import LogNorm
a1 = unpolarized(f, resonance_lineshapes_2)
a= plt.hist2d(m2_12, m2_23, bins=100, weights=a1*weights, norm=LogNorm())
plt.colorbar()
plt.ylabel(r"$M(\Lambda_c\, K)$")
plt.xlabel(r"$M(\Lambda_c \,D^0)$")

In [ ]:
a= plt.hist2d(m2_13, m2_23, bins=100, weights=a1, norm=LogNorm())
plt.colorbar()
plt.ylabel(r"$M(\Lambda_c\, K)$")
plt.xlabel(r"$M(D^0 \, K)$")